In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 200 kB 63.2 MB/s 
     |████████████████████████████████| 199 kB 17.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=091438b5150dfe34d5e3323e7fa3ab45f59bb6767493b5fe4c57a85395154fd0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


#### Logistic Regression Implementation in Pyspark Example

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()
from pyspark.ml.classification import LogisticRegression
my_data = spark.read.format('libsvm').load('/content/drive/MyDrive/Colab Notebooks/data/sample_libsvm_data.txt')

In [3]:
my_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [4]:
my_log_reg_model = LogisticRegression()
fitted_logreg = my_log_reg_model.fit(my_data)
log_summary = fitted_logreg.summary
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [5]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [7]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)
prediction_and_labels = fit_final.evaluate(lr_test)

In [8]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[28.4812199905992...|[0.99999999999957...|       0.0|
|  0.0|(692,[123,124,125...|[37.2251848642471...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[35.6452646502206...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[37.6885163012990...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[17.8329250516942...|[0.99999998200055...|       0.0|
|  0.0|(692,[125,126,127...|[13.2118337667480...|[0.99999817117266...|       0.0|
|  0.0|(692,[126,127,128...|[17.7050130372382...|[0.99999997954447...|       0.0|
|  0.0|(692,[126,127,128...|[28.6800680813226...|[0.99999999999964...|       0.0|
|  0.0|(692,[127,128,129...|[29.3639795221566...|[0.99999999999982...|       0.0|
|  0.0|(692,[150

In [9]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
my_eval = BinaryClassificationEvaluator()
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)
my_final_roc

0.9592592592592591